In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 37193668
paper_name = 'schulze_tamas_2023' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Suppl-data_Table_S3(1).xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 248 x 17


In [7]:
original_data.head()

,Al 2.4 mM,Gene,Description,Unnamed: 3,Unnamed: 4,Unnamed: 5,Ga 5.0 mM,Gene.1,Description.1,Unnamed: 9,Unnamed: 10,Unnamed: 11,In 2.6 mM,Gene.2,Description.2,Unnamed: 15,Unnamed: 16
0,YEL007W,MIT1,Transcriptional regulator of pseudohyphal growth,ORF,MIT1,NaN,YKL157W,APE2,Aminopeptidase yscII,ORF,APE2,NaN,YEL007W,MIT1,Transcriptional regulator of pseudohyphal growth,ORF,MIT1
1,YNL236W,SIN4,Subunit of the RNA polymerase II mediator complex,ORF,SIN4,NaN,YOL009C,MDM12,"Mitochondrial outer membrane protein, ERMES co...",ORF,MDM12,NaN,YDR245W,MNN10,Subunit of a Golgi mannosyltransferase complex,ORF,MNN10
2,YKL157W,APE2,Aminopeptidase yscII,ORF,APE2,NaN,YOL085C,0,Putative protein of unknown function,ORF,YOL085C,NaN,YDR359C,EAF1,Component of the NuA4 histone acetyltransferas...,ORF,EAF1
3,YDR245W,MNN10,Subunit of a Golgi mannosyltransferase complex,ORF,MNN10,NaN,YPR173C,VPS4,AAA-ATPase involved in multivesicular body (MV...,ORF,VPS4,NaN,YML052W,SUR7,"Plasma membrane protein, component of eisosomes",ORF,SUR7
4,YDR359C,EAF1,Component of the NuA4 histone acetyltransferas...,ORF,EAF1,NaN,YJR090C,GRR1,F-box protein component of an SCF ubiquitin-li...,ORF,GRR1,NaN,YML047C,PRM6,Potassium transporter that mediates K+ influx,ORF,PRM6


In [8]:
hit_cols = ['Al 2.4 mM','Ga 5.0 mM', 'In 2.6 mM']
orfs = []
for c in hit_cols:
    orfs = orfs + list(original_data[c])
orfs = list(set(orfs))
len(orfs)

352

In [9]:
orfs.remove(np.nan)

In [10]:
data = pd.DataFrame(index=orfs, columns=hit_cols, data=0)
for c in hit_cols:
    this_orfs = list(original_data[c].values)
    this_orfs = [orf for orf in this_orfs if not str(orf) == 'nan']
    data.loc[this_orfs,c] = -1

In [11]:
data.sum(axis=0)

Al 2.4 mM   -171
Ga 5.0 mM   -133
In 2.6 mM   -248
dtype: int64

In [12]:
data = data.reset_index()

In [13]:
data.head()

,index,Al 2.4 mM,Ga 5.0 mM,In 2.6 mM
0,YGL168W,-1,-1,-1
1,YLR021W,0,0,-1
2,YML097C,-1,0,0
3,YNL127W,0,0,-1
4,YBL088C,0,-1,0


In [14]:
data['orf'] = data['index'].astype(str)

In [15]:
# Eliminate all white spaces & capitalize
data['orf'] = clean_orf(data['orf'])

In [16]:
# Translate to ORFs 
data['orf'] = translate_sc(data['orf'], to='orf')

In [17]:
data = data.groupby(data['orf']).mean()

In [18]:
data = data.reset_index()

In [19]:
# Make sure everything translated ok
t = looks_like_orf(data['orf'])
print(data.loc[~t,])

Empty DataFrame
Columns: [orf, Al 2.4 mM, Ga 5.0 mM, In 2.6 mM]
Index: []


In [20]:
data.set_index('orf', inplace=True)

In [21]:
data.shape

(351, 3)

# Load & process tested strains

In [22]:
tested = pd.read_excel('raw_data/data.xlsx', sheet_name='data')

In [23]:
tested['orf'] = tested['ORF'].astype(str)

In [24]:
tested['orf'] = clean_orf(tested['orf'])

In [25]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [26]:
tested = tested.loc[~(tested['orf']=='WT')]

In [27]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [ORF, Al08, Ga45, In09, metal0, In26, Ga5, Al24, orf]
Index: []


In [28]:
tested_orfs = tested['orf'].unique()

In [29]:
missing = [orf for orf in data.index.values if orf not in tested_orfs]
missing

[]

In [30]:
tested_orfs.shape

(4268,)

In [31]:
data = data.reindex(index=tested_orfs, fill_value=0)

In [32]:
data.shape

(4268, 3)

In [33]:
data.head()

,Al 2.4 mM,Ga 5.0 mM,In 2.6 mM
orf,,,
YAL002W,-1,0,-1
YAL004W,0,0,0
YAL005C,0,0,0
YAL007C,0,0,0
YAL008W,0,0,-1


# Prepare the final dataset

In [34]:
dataset_ids = [22243, 22247, 22245]
datasets = datasets.reindex(index=dataset_ids)

In [35]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [36]:
data.head()

dataset_id,22243,22247,22245
data_type,value,value,value
orf,,,
YAL002W,-1,0,-1
YAL004W,0,0,0
YAL005C,0,0,0
YAL007C,0,0,0
YAL008W,0,0,-1


## Subset to the genes currently in SGD

In [37]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [38]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22243,22247,22245
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,-1,0,-1
1863,YAL004W,0,0,0
4,YAL005C,0,0,0
5,YAL007C,0,0,0
6,YAL008W,0,0,-1


# Normalize

In [39]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [40]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [41]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,22243,22247,22245,22243,22247,22245
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,-1,0,-1,-4.983012,0.0,-4.137749
1863,YAL004W,0,0,0,0.000000,0.0,0.000000
4,YAL005C,0,0,0,0.000000,0.0,0.000000
5,YAL007C,0,0,0,0.000000,0.0,0.000000
6,YAL008W,0,0,-1,0.000000,0.0,-4.137749


# Print out

In [42]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')